In [137]:
import pandas as pd
import time

from transformers import *
import torch
from torch.utils.data import TensorDataset, DataLoader

In [140]:
data_path = 'context_task_data.tsv'
data = pd.read_csv(data_path, sep='\t')[0:80]

In [3]:
# data

In [135]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-cased', 'bert-base-cased') # initialize Bert2Bert


In [147]:
n_train = 60
x_train = data.TaskSentence.values[:n_train]
y_train = data.Summary.values[:n_train]

In [148]:
# input_ids_list = torch.tensor(tokenizer.batch_encode_plus(x_train, pad_to_max_length=True)['input_ids'])
# lm_labels_list = torch.tensor(tokenizer.batch_encode_plus(y_train, pad_to_max_length=True)['input_ids'])

input_ids_list = torch.tensor(
    [tokenizer.encode(x, pad_to_max_length=True, add_special_tokens=True) for x in x_train])
lm_labels_list = torch.tensor(
    [tokenizer.encode(y, pad_to_max_length=True, add_special_tokens=True) for y in y_train])

dataset = TensorDataset(input_ids_list, lm_labels_list)
loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=1
)

In [146]:
input_ids_list[:2]

tensor([[  101,  4203,  1977,  ...,     0,     0,     0],
        [  101,  4203, 10621,  ...,     0,     0,     0]])

In [151]:
bptt = 5
def get_batch(x_train, y_train, i):
    seq_len = min(bptt, len(x_train)-1-i)
    x = x_train[i:i+seq_len]
    y = y_train[i:i+seq_len]
    return x,y

In [161]:
input_ids_list.size().numpy()

AttributeError: 'torch.Size' object has no attribute 'numpy'

In [162]:
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(epoch):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    
    for batch, i in enumerate(range(0, input_ids_list.size(0) - 1, bptt)):
        x, y = get_batch(input_ids_list, lm_labels_list, i)
        optimizer.zero_grad()
        loss, outputs = model(input_ids=x, decoder_input_ids=x, lm_labels=y)[:2]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 1
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(x_train) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


In [166]:
# train(1)

In [167]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [168]:
count_parameters(model)

217241668

In [121]:
# n_epochs = 10
# lr = 5.0 # learning rate
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# for _ in n_eopchs:
    
#     for x, y in loader:
#         optimizer.zero_grad()
#         loss, outputs = model(input_ids=x, decoder_input_ids=x, lm_labels=y)[:2]
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
#         optimizer.step()

In [133]:
tokenizer.decode(generated[0])

'[PAD].. sex sex sexiteiteiteiteiteiteiteiteiteiteiteiteiteite'

In [169]:
model2 = T5ForConditionalGeneration.from_pretrained("t5-base")

In [170]:
count_parameters(model2)

222903936